<a href="https://colab.research.google.com/github/mskbasha/Sentiment_analyis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import torch 
import math
import pandas as pd 
import numpy as np
from torch.nn import functional as F

# Reading Data

In [1]:

class MultiHeadedAttention(torch.nn.Module):
    def __init__(self,heads,seq_length,dim):
        super().__init__()
        assert dim % heads==0
        self.heads = heads
        self.dim = dim
        self.qkv_dim = heads//dim
        self.ff = torch.nn.Linear(self.dim,self.dim,bias = False)
        self.MHA = torch.nn.MultiheadAttention(dim,heads,batch_first=True)
        self.o_proj = torch.nn.Linear(dim,dim,bias = False)
        self.layernorm = torch.nn.LayerNorm([seq_length,dim])
    def __call__(self,x,pad_mask=None,future_mask=None):
        self.q = self.ff(x)
        self.k = self.ff(x)
        self.v = self.ff(x)
        output,_ = self.MHA(self.q,self.k,self.v,key_padding_mask=pad_mask,attn_mask=future_mask)
        print(output)
        return self.layernorm(output)
class Encoder(torch.nn.Module):
    def __init__(self,no_blocks,heads,seq_length,dim):
        super().__init__()
        self.no_blocks = no_blocks
        self.MHA = MultiHeadedAttention(heads,seq_length,dim)
        self.ff = torch.nn.Linear(dim,dim)
    def __call__(self,x,pad_mask = None):
        
        for i in range(self.no_blocks):
            output = self.MHA(x,pad_mask=pad_mask)
            output += self.ff(x+output)
        return output
class Decoder(torch.nn.Module):
    def __init__(self,no_blocks,heads,seq_length,dim,future_mask=None):
        super().__init__()
        self.seq_length = seq_length
        self.no_blocks = no_blocks
        self.MHA = MultiHeadedAttention(heads,seq_length,dim)
        self.ff = torch.nn.Linear(dim,dim)
    def _create_mask(self,iteration): 
        mask = torch.ones(self.seq_length, self.seq_length)
        mask = mask.triu_(diagonal=1 + iteration)
        return mask
    def __call__(self,x,future_mask=None):
        future_mask = self._create_mask(1)
        for i in range(self.no_blocks):
            output = self.MHA(x,future_mask=future_mask)
            output += self.ff(x+output)
        return output
    def _create_mask(self,x):
        pass
# class Transformer()
x = Decoder( 1,2,5,6)(torch.normal(0,1,size = (2,5,6)))
x

tensor([[[-0.0468,  0.1302, -0.1449,  0.1121,  0.0797, -0.1255],
         [-0.0531,  0.1255, -0.1361,  0.1127,  0.0835, -0.1183],
         [-0.0500,  0.1196, -0.1442,  0.1054,  0.0788, -0.1154],
         [-0.0679,  0.1181, -0.1161,  0.1096,  0.0881, -0.1065],
         [-0.0683,  0.1294, -0.1511,  0.1240,  0.0792, -0.1091]],

        [[-0.1219,  0.2945, -0.1813,  0.2992,  0.1208, -0.1694],
         [-0.1344,  0.1958, -0.1305,  0.2183,  0.1182, -0.0980],
         [-0.0747,  0.2288, -0.1483,  0.2236,  0.1078, -0.1475],
         [-0.1659,  0.3574, -0.2101,  0.3709,  0.1316, -0.1907],
         [-0.1284,  0.1644, -0.1087,  0.1874,  0.1141, -0.0807]]],
       grad_fn=<TransposeBackward0>)


tensor([[[ 0.9797,  1.7493, -1.7915,  0.8403,  0.0917, -1.8617],
         [ 0.0115,  1.1668, -1.6763,  1.1441,  0.7377, -1.7793],
         [ 0.1067,  0.7168, -0.8846,  1.0514,  0.7862, -1.0022],
         [-1.3754, -0.3129, -0.2582,  0.9532,  1.3801, -1.0791],
         [-0.8339,  0.4875, -0.6378,  1.8469,  0.5473, -0.6652]],

        [[-0.2774,  1.2891, -0.9990,  2.1361,  0.4516, -0.8229],
         [-1.4020, -0.2505, -0.2756,  0.8289,  0.9758, -1.2685],
         [ 0.3111,  1.2268, -0.4163,  1.4974,  0.1288, -0.3546],
         [-0.4270,  1.6623, -1.1173,  2.6046, -0.1118, -0.8787],
         [-1.2488, -0.3269, -0.0777,  0.3790,  0.8444, -1.0327]]],
       grad_fn=<AddBackward0>)

tensor([[False, False, False, False, False],
        [False, False, False, False, False],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True]])

In [8]:
masked_qkt = torch.tensor([[1,1,0],[2,1,0],[1,3,0]])
values = torch.tensor([[1,2,3,4],[4,5,6,7],[8,9,2,1]])
torch.matmul(masked_qkt,values)

tensor([[ 5,  7,  9, 11],
        [ 6,  9, 12, 15],
        [13, 17, 21, 25]])

NameError: ignored

In [51]:
torch.sqrt(torch.tensor([1,2,3,4,5])).masked_fill(torch.tensor(  [True,True,False,False,False])   , float('-inf'))

tensor([  -inf,   -inf, 1.7321, 2.0000, 2.2361])

In [81]:
k = torch.tensor([[
    [1,2,0],
    [3,0,5],
    [6,0,1],
    [0,1,2],
],
[
    [1,2,0],
    [3,0,5],
    [6,0,1],
    [0,1,2],
]])
torch.cat([k[0,:,:],k[1,:,:]],dim=1)

tensor([[1, 2, 0, 1, 2, 0],
        [3, 0, 5, 3, 0, 5],
        [6, 0, 1, 6, 0, 1],
        [0, 1, 2, 0, 1, 2]])

In [60]:
torch.normal(0,1,size=(4,3,3,3)).masked_fill(k[:,None,None,:]==0,float('-inf')).shape

torch.Size([4, 3, 3, 3])

In [61]:
torch.matmul(torch.normal(0,1,size=(10,5,10,10)),torch.normal(0,1,size = (10,5,10,5))).shape

torch.Size([10, 5, 10, 5])

In [64]:
torch.tensor([[1,2,3],[3,4,5]])

tensor([[1, 2, 3],
        [3, 4, 5]])

In [67]:
torch.tensor([[1,2,3],[3,4,5]]).reshape(3,2)

tensor([[1, 2],
        [3, 3],
        [4, 5]])

In [3]:
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention,LayerNormalization,Dense
import tensorflow as tf

def encoder_block(input_tensor,emb_dim,num_heads):
    q, k, v= Dense(emb_dim)(input_tensor), Dense(emb_dim)(input_tensor), Dense(emb_dim)(input_tensor)
    output = MultiHeadAttention(num_heads=num_heads, key_dim=emb_dim)(q,k,v)
    output = LayerNormalization()(output)
    res = input_tensor+output
    output = Dense(emb_dim)(res)
    output = output+res
    return q,k,v,output
input_tensor = tf.keras.layers.Input((5,12))
output = input_tensor
for i in range(10): 
    q,k,v, output = encoder_block(output,12,2)

model = keras.Model(inputs=input_tensor, outputs=output)

In [4]:
k = tf.random.normal((10,5,12),0,1)
model.predict(k).shapex

1/1 [==============================] - 2s 2s/step


(10, 5, 12)

In [102]:
import keras
keras.layers.Dense(1)(tf.random.normal((10,10,),0,1)).shape

TensorShape([10, 1])